# Training Dataset Preprocessing

From the analysis part, it is concluded that the following variables should be included in the dataset:

**Numerical Features**:
1. `amount_tsh`
2. `gps_height`
3. `population`
4. `season`
5. `operation_time`
6. `public_meeting`
7. `permit`

**Categorical Features**
1. `funder`
2. `installer`
3. `basin`
4. `lga`
5. `region_code`
6. `district_code`
7. `basin`
8. `scheme_management`
9. `management`
10. `payment`
11. `extraction_type`
12. `source`
13. `water_quality`
14. `quantity`

**Total number of features: 19**

# Importing Training Samples

In [19]:
import pandas as pd

samples = pd.read_csv('train.csv')
targets = pd.read_csv('targets.csv')

samples

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


# Amount TSH, Population, GPS Height

The `amount_tsh` column can be imputed using the mean value of pumps from the same subvillage.
However, since the `gps_height` is location dependent, it imputed by also using wards and even longitude and latitude, in order to find nearby pumps.
The `population` will be imputed just like amount_tsh column.

**Helper Functions**:
1. `impute_column_by_nearby_samples`: Imputes a specified column using the mean of nearby located pumps
2. `find_k_nearest_by_coordinates`: Returns the k nearest located pumps, using the longitude & latitude

In [20]:
import numpy as np


def find_k_nearest_by_coordinates(
        samples_df: pd.DataFrame,
        k_nearest: int,
        longitude: float,
        latitude: float
) -> pd.DataFrame:
    geo_coordinates = samples_df[['longitude', 'latitude']]
    target_coord = np.float32([longitude, latitude])
    distances = np.sqrt(np.sum(np.power(target_coord - geo_coordinates, 2), axis=1))
    min_distance_indices = np.argpartition(distances, k_nearest)[1: k_nearest+1]
    return samples_df.iloc[min_distance_indices]


def impute_zeros_by_nearby_samples(
        samples_df: pd.DataFrame,
        location_column: str,
        target_column: str,
        std_threshold: float or None
) -> int:
    num_imputed = 0

    for area in samples_df[location_column].unique():
        row_ids = samples_df[location_column] == area
        target_values = samples_df.loc[row_ids, target_column]

        if target_values.shape[0] > 1:
            non_zero_ids = target_values > 0

            if non_zero_ids.sum() > 0:
                non_zero_values = target_values[non_zero_ids]

                if std_threshold is not None and np.std(non_zero_values) > std_threshold:
                    continue

                zero_ids = np.invert(non_zero_ids)
                target_values[zero_ids] = non_zero_values.mean()
                samples_df.loc[row_ids, target_column] = target_values
                num_imputed += zero_ids.sum()
    return num_imputed

In [21]:
amount_tsh_std_threshold = 50

num_amount_tsh_missing = (samples["amount_tsh"] == 0).sum()
num_imputed = impute_zeros_by_nearby_samples(
    samples_df=samples,
    location_column='subvillage',
    target_column='amount_tsh',
    std_threshold=amount_tsh_std_threshold
)
f'Imputed {num_imputed}/{num_amount_tsh_missing} missing "amount_tsh" values'

'Imputed 4573/41639 missing "amount_tsh" values'

In [22]:
population_std_threshold = 50

num_amount_tsh_missing = (samples["population"] == 0).sum()
num_imputed = impute_zeros_by_nearby_samples(
    samples_df=samples,
    location_column='subvillage',
    target_column='population',
    std_threshold=population_std_threshold
)
f'Imputed {num_imputed}/{num_amount_tsh_missing} missing "population" values'

'Imputed 1363/21381 missing "population" values'

In [23]:
location_columns = ['subvillage', 'ward', 'lga', 'district_code']

for location_column in location_columns:
    num_gps_height_missing = (samples["gps_height"] == 0).sum()
    num_imputed = impute_zeros_by_nearby_samples(
        samples_df=samples,
        location_column=location_column,
        target_column='gps_height',
        std_threshold=None
    )
    print(f'Imputed {num_imputed}/{num_gps_height_missing} missing "gps_height" values using "subvillages"')

Imputed 5354/20438 missing "gps_height" values using "subvillages"
Imputed 13933/15804 missing "gps_height" values using "subvillages"
Imputed 2558/2604 missing "gps_height" values using "subvillages"
Imputed 89/89 missing "gps_height" values using "subvillages"


In [24]:
k_neighbors = 25
gps_height_zero_ids = samples['gps_height'] == 0
gps_zero_samples = samples[gps_height_zero_ids]
gps_heights = []

for _, sample in gps_zero_samples.iterrows():
    longitude = sample['longitude']
    latitude = sample['latitude']
    nearest_samples = find_k_nearest_by_coordinates(
        samples_df=samples,
        k_nearest=k_neighbors,
        longitude=longitude,
        latitude=latitude
    )
    non_zero_gps_height_values = nearest_samples.loc[nearest_samples['gps_height'] != 0, 'gps_height']
    gps_heights.append(non_zero_gps_height_values.mean())

samples.loc[samples['gps_height'] == 0, 'gps_height'] = gps_heights
f'gps_height == 0: {(samples["gps_height"] == 0).sum()} After K-NN method'

'gps_height == 0: 0 After K-NN method'

# Computing Recorded Season & Operation Time

| ID  | Season     | Months               |
|-----|------------|----------------------|
| 0   | ShortDry   | January - February   |
| 1   | LongRains  | March - May          |
| 2   | LongDry    | June - October       |
| 3   | ShortRains | November - December  |

The operation time is computed as: $Operation Time = Year Recorded - Construction Year$
All invalid operation times should be assigned: `-1`.

In [25]:
seasons = {
    1: 0, 2: 0,
    3: 1, 4: 1, 5: 1,
    6: 2, 7: 2, 8: 2, 9: 2, 10: 2,
    11: 3, 12: 3,
}

samples['seasons'] = pd.DataFrame({'Months': pd.DatetimeIndex(samples['date_recorded']).month})
samples['seasons'] = samples['seasons'].apply(lambda month: seasons[month])
samples['seasons'].value_counts()

1    22242
0    18756
2    16432
3     1970
Name: seasons, dtype: int64

In [26]:
samples['construction_year'] = samples['construction_year'].replace({0: 10000})
samples['operation_time'] = pd.DatetimeIndex(samples['date_recorded']).year - samples['construction_year']

invalid_operation_time_ids = samples['operation_time'] < 0
samples.loc[invalid_operation_time_ids, 'operation_time'] = -1

f'{invalid_operation_time_ids.sum()} invalid dates were set to -1'

'20718 invalid dates were set to -1'

# Funder & Installer

1. "`Nans`" will be converted to "Unknown" category
2. Funders with low appearance frequency or numerical values will be assigned the "`other`" value
3. The **Monge-Elkan** similarity score will be used to correctly match misspelled or similar tokens
4. A low threshold (e.g. 50) should be used, so similar tokens are grouped together (e.g. **Germany Government** & **Tanzania Government** --> **Government**)

**Helper Functions:**
1. `preprocess_single_token`: Removes duplicates, stop-words and multiple whitespaces and capitals from a token.
2. `preprocess_tokens`: Preprocesses the tokens of an entire column.
3. `cluster_similar_tokens`: Clusters similar tokens using a Monge-Elkan algorithm.
In case 2 tokens are matched, the token with the highest counts becomes the cluster's representative.

In [27]:
from py_stringmatching.similarity_measure.monge_elkan import MongeElkan
import re

non_allowed = ['.', '/', '\'', ',', '&', '(', ')', '-', '_', '1', '2', '3', '4', '5', '6', '7', '8', '9']
similarity_algorithm = MongeElkan()


def preprocess_single_token(token: str) -> str:
    for ch in non_allowed:
        token = token.replace(ch, ' ')
    token = re.sub('\\s+', ' ', token).lower()
    return token.lower()


def preprocess_tokens(value_counts: pd.Series) -> dict:
    preprocessed_tokens = {}

    tokens = list(value_counts.index)
    counts = list(value_counts.values)

    for token, count in zip(tokens, counts):
        processed_token = preprocess_single_token(token=token)

        if not processed_token in preprocessed_tokens or preprocessed_tokens[processed_token] < count:
            preprocessed_tokens[processed_token] = count
    return preprocessed_tokens

def cluster_similar_tokens(
        samples_df: pd.DataFrame,
        target_column: str,
        similarity_threshold: int
) -> (pd.DataFrame, int):
    clusters = {}
    num_clustered = 0

    original_token_counts = samples_df[target_column].value_counts()

    preprocessed_token_with_counts = preprocess_tokens(value_counts=original_token_counts)
    preprocess_token_list = list(preprocessed_token_with_counts.keys())

    for original_token, original_token_count in original_token_counts.items():
        preprocessed_token = preprocess_single_token(token=original_token)

        similarities = {
            token2: similarity_algorithm.get_raw_score(preprocessed_token.split(' '), token2.split(' '))
            for token2 in preprocess_token_list if preprocessed_token != token2
        }

        most_similar_token = max(similarities, key=similarities.get)
        similarity_score = similarities[most_similar_token]

        if similarity_score >= similarity_threshold and original_token_count < preprocessed_token_with_counts[most_similar_token]:
            clusters[original_token] = most_similar_token
            num_clustered += 1
        else:
            clusters[original_token] = preprocessed_token

    samples_df[target_column] = samples[target_column].apply(lambda token: clusters[token])
    return samples_df, num_clustered

In [28]:
print(f'Number of unique funders = {samples["funder"].unique().shape[0]} before clustering')

similarity_threshold = 0.5

samples['funder'] = samples['funder'].fillna('unknown')
samples['funder'] = samples['funder'].replace({'0': 'other', '-1': 'other'})

num_clustered = 1000
iteration = 0

while num_clustered > 0:
    samples, num_clustered = cluster_similar_tokens(
        samples_df=samples,
        target_column='funder',
        similarity_threshold=similarity_threshold
    )

    iteration += 1
    print(f'Iteration = {iteration}, Clustered {num_clustered} tokens')

for funder, count in samples['funder'].value_counts().items():
    if count < 10:
        samples.loc[samples['funder'] == funder, 'funder'] = 'other'

f'Number of unique funders = {samples["funder"].unique().shape[0]} after clustering'

Number of unique funders = 1898 before clustering
Iteration = 1, Clustered 798 tokens
Iteration = 2, Clustered 290 tokens
Iteration = 3, Clustered 124 tokens
Iteration = 4, Clustered 49 tokens
Iteration = 5, Clustered 24 tokens
Iteration = 6, Clustered 4 tokens
Iteration = 7, Clustered 1 tokens
Iteration = 8, Clustered 0 tokens


'Number of unique funders = 233 after clustering'

In [29]:
f'Number of unique installers = {samples["installer"].unique().shape[0]} before clustering'

similarity_threshold = 0.5

samples['installer'] = samples['installer'].fillna('unknown')
samples['installer'] = samples['installer'].replace({'0': 'other', '-1': 'other'})

num_clustered = 1000
iteration = 0

while num_clustered > 0:
    samples, num_clustered = cluster_similar_tokens(
        samples_df=samples,
        target_column='installer',
        similarity_threshold=similarity_threshold
    )

    iteration += 1
    print(f'Iteration = {iteration}, Clustered {num_clustered} tokens')

for installer, count in samples['installer'].value_counts().items():
    if count < 10:
        samples.loc[samples['installer'] == installer, 'installer'] = 'other'

f'Number of unique installers = {samples["installer"].unique().shape[0]} after clustering'

Iteration = 1, Clustered 952 tokens
Iteration = 2, Clustered 306 tokens
Iteration = 3, Clustered 138 tokens
Iteration = 4, Clustered 57 tokens
Iteration = 5, Clustered 16 tokens
Iteration = 6, Clustered 6 tokens
Iteration = 7, Clustered 3 tokens
Iteration = 8, Clustered 1 tokens
Iteration = 9, Clustered 1 tokens
Iteration = 10, Clustered 0 tokens


'Number of unique installers = 226 after clustering'

# Public Meeting & Permit

Convert True/False/Nan to 1/0/-1

In [30]:
samples['public_meeting'] = samples['public_meeting'].fillna('Unknown')
samples['public_meeting'] = samples['public_meeting'].replace({'False': 0, 'True': 1, 'Unknown': -1})

f'Assigned {(samples["public_meeting"] == -1).sum()} Nans values of "public_meeting" column to -1'

'Assigned 3334 Nans values of "public_meeting" column to -1'

In [31]:
samples['permit'] = samples['permit'].fillna('Unknown')
samples['permit'] = samples['permit'].replace({'False': 0, 'True': 1, 'Unknown': -1})
f'Assigned {(samples["permit"] == -1).sum()} Nans values of "permit" column to -1'

'Assigned 3056 Nans values of "permit" column to -1'

# Scheme Management

1. Replace the unique `None` value with the most frequent value of the same subvillage or ward
2. Convert the remaining `NaNs` to `unknown` category

**Helper Functions**
1. `assign_most_frequent_by_nearby_samples`: It assigns the most frequent value of a specified column to the missing values using nearby samples,
if the frequent value is appeared above a threshold.

In [32]:
def impute_nans_by_nearby_samples(
        samples_df: pd.DataFrame,
        location_column: str,
        target_column: str,
        frequency_threshold: float or None
) -> (pd.DataFrame, int):
    num_imputed = 0

    for area in samples_df[location_column].unique():
        row_ids = samples_df[location_column] == area
        target_values = samples_df.loc[row_ids, target_column]

        if target_values.shape[0] > 1:
            not_na_ids = target_values.notna()

            if not_na_ids.sum() > 0:
                value_frequencies = target_values.value_counts() / target_values.shape[0]
                non_nan_values = target_values[not_na_ids]
                most_frequent_value = non_nan_values.mode()

                if frequency_threshold is not None and value_frequencies[most_frequent_value].tolist()[0] < frequency_threshold:
                    continue

                nan_ids = np.invert(not_na_ids)
                target_values[nan_ids] = most_frequent_value
                samples_df.loc[row_ids, target_column] = target_values
                num_imputed += nan_ids.sum()
    return samples_df, num_imputed

In [33]:
frequency_threshold = 0.7

for location in ['subvillage', 'ward']:
    num_nans = samples['scheme_management'].isna().sum()

    samples, num_imputed = impute_nans_by_nearby_samples(
        samples_df=samples,
        location_column=location,
        target_column='scheme_management',
        frequency_threshold=frequency_threshold
    )
    print(f'Imputed {num_imputed}/{num_nans} of "scheme_management" values using "{location}" location')

num_nans = samples['scheme_management'].isna().sum()
samples['scheme_management'] = samples['scheme_management'].fillna('unknown')
print(f'NaNs converted to "Unknown" = {num_nans}')

Imputed 255/3877 of "scheme_management" values using "subvillage" location
Imputed 443/3877 of "scheme_management" values using "ward" location
NaNs converted to "Unknown" = 3877


# Management, Payment Type, Water Quality, Quantity

Replace unknowns with the most frequent value in each subvillage/ward if it exceeds a frequency threshold

In [34]:
def impute_unknowns_by_nearby_samples(
        samples_df: pd.DataFrame,
        location_column: str,
        target_column: str,
        frequency_threshold: float
) -> (pd.DataFrame, int):
    num_imputed = 0

    for area in samples_df[location_column].unique():
        row_ids = samples_df[location_column] == area
        target_values = samples_df.loc[row_ids, target_column]

        if target_values.shape[0] > 1:
            not_unknown_ids = target_values != 'unknown'

            if not_unknown_ids.sum() > 0:
                value_frequencies = target_values.value_counts() / target_values.shape[0]
                most_frequent_value_frequency = value_frequencies.values[0]

                if value_frequencies.index[0] != 'unknown' and most_frequent_value_frequency >= frequency_threshold:
                    unknown_ids = np.invert(not_unknown_ids)
                    target_values[unknown_ids] = value_frequencies.index[0]
                    samples_df.loc[row_ids, target_column] = target_values
                    num_imputed += unknown_ids.sum()
    return samples_df, num_imputed

In [35]:
frequency_threshold = 0.8

for location_col in ['subvillage', 'ward']:
    for target_col in ['management', 'payment_type', 'water_quality', 'quantity']:
        num_unknowns = (samples[target_col] == 'unknown').sum()

        samples, num_imputed = impute_unknowns_by_nearby_samples(
            samples_df=samples,
            location_column=location_col,
            target_column=target_col,
            frequency_threshold=frequency_threshold
        )
        print(f'Imputed {num_imputed}/{num_unknowns} "unknown" values of column "{target_col}" using nearby "{location_col}" samples')

Imputed 56/561 "unknown" values of column "management" using nearby "subvillage" samples
Imputed 89/8157 "unknown" values of column "payment_type" using nearby "subvillage" samples
Imputed 345/1876 "unknown" values of column "water_quality" using nearby "subvillage" samples
Imputed 42/789 "unknown" values of column "quantity" using nearby "subvillage" samples
Imputed 177/505 "unknown" values of column "management" using nearby "ward" samples
Imputed 131/8068 "unknown" values of column "payment_type" using nearby "ward" samples
Imputed 298/1531 "unknown" values of column "water_quality" using nearby "ward" samples
Imputed 73/747 "unknown" values of column "quantity" using nearby "ward" samples


# Storing Train Dataset

In [36]:
samples.to_csv('train_inputs.csv', index=False)
samples

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,seasons,operation_time
0,69572,6000.0,2011-03-14,roman catholic,1390.000000,roman cathoric kilomeni,34.938093,-9.856322,none,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1,12
1,8776,0.0,2013-03-06,rudep,1399.000000,grumeti,34.698766,-2.147466,Zahanati,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1,3
2,34310,25.0,2013-02-25,rotary club,686.000000,world vision,37.460664,-3.821329,Kwa Mahundi,0,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,0,4
3,67743,0.0,2013-01-28,unicef,263.000000,unicef,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,0,27
4,19728,0.0,2011-07-13,mzinga a,1300.000000,artisan,31.130847,-1.825359,Shuleni,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,germany republi,1210.000000,ces,37.169807,-3.253847,Area Three Namba 27,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1,14
59396,27263,4700.0,2011-05-07,cefa njombe,1212.000000,rcchurch cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,1,15
59397,37057,0.0,2011-04-11,unknown,867.728155,unknown,34.017087,-8.750434,Mashine,0,...,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,1,-1
59398,31282,50.0,2011-03-08,other,401.400000,muwsa,35.861315,-6.378573,Mshoro,0,...,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,1,-1
